In [25]:
import os, clipboard
pj = os.path.join
refd = "D:\\auto_hd_install\\default configs\\IM00226 Rev C\\Mag 03"
lsf = pj(refd, "Logging On.log")
ls = set()
with open(lsf, 'r') as f:
    f.readline()  # header
    txt = f.read().splitlines()
for line in txt:
    var, _ = line.split("\t", 1)
    ls.add(var)

In [87]:
AUTO = 0
MAN = 1
OFF = 2

SEC = 1
MIN = SEC * 60
HOUR = MIN * 60
DAY = HOUR * 24
MSEC = SEC / 1000

class Recipe():
    def __init__(self, name="Recipe"):
        self.name = name
        self.steps = [""]  # placeholder for name
        self.ls = ls
        self.add = self.steps.append
    
    def validate(self, v):
        if v not in self.ls:
            cs = "\n".join("%-40s: %.1f%%"%(a, b*100) for a,b in closest(v))
            raise ValueError("Invalid variable '%s'. Closest: \n\n%s"%(v, cs))
            
    def translate(self, v):
        return str(v).upper()
            
    def set(self, var, val):
        self.validate(var)
        val = self.translate(val)
        l = "Set \"%s\" to %s" % (var, val)
        self.add(l)
    
    def wait(self, t):
        self.add("Wait %d seconds"%(int(t)))
    
    def wait_until(self, var, op, val):
        self.validate(var)
        val = self.translate(val)
        self.add("Wait until \"%s\" %s %s"%(var, op, val))
        
    def finish(self):
        self.steps[0] = "Func %s"%self.name
        return "\n".join(self.steps)

In [83]:
def closest(v):  # original
    l = ls
    vl = len(v)
    for i in range(len(v)):
        l2 = set()
        c = v[i]
        if v in ls:
            return [v]
        for var in l:
            try:
                c2 = var[i]
            except IndexError:
                continue
            if c == c2:
                l2.add(var)
        if not l2:
            break
        l = l2
    return sorted(l)
import difflib
def ratio(a,b):
    return difflib.SequenceMatcher(None, a,b).ratio()
def closest(v):
    scores = []
    for var in ls:
        s = ratio(v, var)
        scores.append((s, var))
    scores = sorted(scores, reverse=True)[:5]
    return [(b,a) for a,b in scores]
                

In [89]:
def recipe(name):
    global __recipe
    global wait
    global setv
    global wait_until
    global finish
    __recipe = Recipe(name)
    wait = __recipe.wait
    setv = __recipe.set
    wait_until = __recipe.wait_until
    def finish(copy=False):
        s = __recipe.finish()
        print(s)
        if copy:
            clipboard.copy(s)
    
def sethelp(mode, val, modev, autov, manv):
    setv(modev,mode)
    if mode == AUTO:
        setv(autov, val)
    elif mode == MAN:
        setv(manv, val)
        
def setag(mode, val=0):
    sethelp(mode, val, "AgModeUser", "AgSP(RPM)", "AgPowerUser(%)")
    
def settemp(mode, val=0):
    sethelp(mode, val, "TempModeUser", "TempSP(C)", "TempHeatDutyUser(%)")

In [90]:
recipe("OQ-AgVerif")
setag(AUTO, 20)
wait(60)
setag(MAN, 60)
wait(30)
setag(OFF)
wait(20)
setag(MAN, 100)
finish()

Func OQ-AgVerif
Set "AgModeUser" to 0
Set "AgSP(RPM)" to 20
Wait 60 seconds
Set "AgModeUser" to 1
Set "AgPowerUser(%)" to 60
Wait 30 seconds
Set "AgModeUser" to 2
Wait 20 seconds
Set "AgModeUser" to 1
Set "AgPowerUser(%)" to 100
